**Install Requirements**

In [0]:
"""!pip3 install 'torch==1.4.0'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'"""

"!pip3 install 'torch==1.4.0'\n!pip3 install 'torchvision==0.5.0'\n!pip3 install 'Pillow-SIMD'\n!pip3 install 'tqdm'"

In [0]:
import os
"""if not os.path.isdir('./Pacs'):
  !git clone https://github.com/lore-lml/machine-learning2020-hw3.git
  !mv 'machine-learning2020-hw3' 'Pacs'
  !rm './Pacs/hw3.ipynb'
  !rm './Pacs/README.md'"""

import logging
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from pacs_dataset import Pacs
from dann import alexdann
from dann import train_src, test_target, dann_train_src_target

from PIL import Image
from tqdm import tqdm

from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
%matplotlib inline

**Set Arguments**

In [0]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

NUM_CLASSES = 7

BATCH_SIZE = 128      # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                      # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3             # The initial Learning Rate
MOMENTUM = 0.9        # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5   # Regularization, you can keep this at the default

NUM_EPOCHS = 10       # Total number of training epochs (iterations over dataset)
STEP_SIZE = 3         # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.5           # Multiplicative factor for learning rate step-down

ALPHA = 'dynamic'
BASE_FILE_PATH = "RUN_1_LR1e-3_DynAlpha_SGD_SS2_GAMMA03"

**Define Data Preprocessing**

In [0]:
transforms = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # Normalizes tensor with mean and standard deviation
])

**Prepare Dataset**

In [0]:
ROOT = 'Pacs/PACS'

source_data = Pacs(ROOT, transform=transforms, source='photo')
cartoon_data = Pacs(ROOT, transform=transforms, source='cartoon')
sketch_data = Pacs(ROOT, transform=transforms, source='sketch')
target_data = Pacs(ROOT, transform=transforms, source='art_painting')

_, source_labels = source_data.get_img_with_labels()
_, cartoon_labels = cartoon_data.get_img_with_labels()
_, sketch_labels = sketch_data.get_img_with_labels()
_, target_labels = target_data.get_img_with_labels()

print(f"# classes source_data: {len(set(source_labels))}")
print(f"# classes cartoon_data: {len(set(cartoon_labels))}")
print(f"# classes sketch_data: {len(set(sketch_labels))}")
print(f"# classes target_data: {len(set(target_labels))}")
print(f"source_data: {len(source_data)} elements")
print(f"cartoon_data: {len(cartoon_data)} elements")
print(f"sketch_data: {len(sketch_data)} elements")
print(f"target_data: {len(target_data)} elements")

# classes source_data: 7
# classes cartoon_data: 7
# classes sketch_data: 7
# classes target_data: 7
source_data: 1670 elements
cartoon_data: 2344 elements
sketch_data: 3929 elements
target_data: 2048 elements


**Prepare Dataloaders**

In [0]:
def get_data_loaders(dataset, batch_size=BATCH_SIZE):
  return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True), DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=False)
"""
# Photo Source Data Loader
source_dataloader = DataLoader(source_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

# Cartoon and sketch validation Data Loaders
cartoon_dataloader = DataLoader(cartoon_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
cartoon_test_dataloader = DataLoader(cartoon_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)
sketch_dataloader = DataLoader(sketch_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
sketch_test_dataloader = DataLoader(sketch_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)

# Art_painting target Data Loader
target_dataloader = DataLoader(target_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_dataloader = DataLoader(target_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)"""

'\n# Photo Source Data Loader\nsource_dataloader = DataLoader(source_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)\n\n# Cartoon and sketch validation Data Loaders\ncartoon_dataloader = DataLoader(cartoon_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)\ncartoon_test_dataloader = DataLoader(cartoon_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)\nsketch_dataloader = DataLoader(sketch_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)\nsketch_test_dataloader = DataLoader(sketch_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)\n\n# Art_painting target Data Loader\ntarget_dataloader = DataLoader(target_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)\ntest_dataloader = DataLoader(target_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=False)'

**Prepare Network**

In [0]:
def init_cnn_objects(model, lr=LR, step_size=STEP_SIZE, gamma=GAMMA):
  
  # Define loss function
  criterion_1 = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
  criterion_2 = nn.CrossEntropyLoss()
  parameters_to_optimize = model.parameters() # In this case we optimize over all the parameters of AlexNet
  
  optimizer = optim.SGD(parameters_to_optimize, lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  #optimizer = optim.Adam(parameters_to_optimize, lr=lr,amsgrad=True)
  #optimizer = optim.AdamW(parameters_to_optimize, lr=lr,amsgrad=True, weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

  return criterion_1, criterion_2, optimizer, scheduler

**Training**

In [0]:
def simple_train_test(model, source_dataloader, test_dataloader, criterion, optimizer, scheduler, max_epoch=NUM_EPOCHS, file_path=BASE_FILE_PATH, device=DEVICE):
    train_losses = []
    loss_min = -1
    accuracy_max = 0
    
    model = model.to(device)
    cudnn.benchmark
    
    current_step = 0
    for epoch in range(max_epoch):
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, max_epoch, scheduler.get_lr()))
        cumulative_loss, current_step = train_src(model, source_dataloader, optimizer, criterion, current_step, device)
        curr_loss = cumulative_loss / len(source_dataloader)
        train_losses.append(curr_loss)
        """if loss_min == -1 or loss_min > curr_loss:
            loss_min = curr_loss
            torch.save(model, f"{file_path}_best_model.pth")"""
        curr_accuracy = test_target(model, test_dataloader, criterion, device) / float(len(target_data))
        print(f"Current Accuracy: {curr_accuracy}")
        if accuracy_max < curr_accuracy:
          accuracy_max = curr_accuracy
          torch.save(model, f"{file_path}_best_model.pth")
        scheduler.step()
        
    # model = torch.load(f"{file_path}_best_model.pth").to(device)
    #accuracy = test_target(model, test_dataloader, criterion, device) / float(len(target_data))
    #print(f"Accuracy on test set: {accuracy}%")
    return train_losses, accuracy_max

In [0]:
params = {
    "lr": [1e-3, 5e-3],
    "batch_size": [128],
    "epochs": [10],
    "step_size": [2, 3],
    "gamma": [0.5],
    "alpha": ['dynamic']
}

grid = ParameterGrid(params)
results = []
for config in grid:
  source_dataloader, _ = get_data_loaders(source_data, config['batch_size'])
  _, cartoon_test_dataloader = get_data_loaders(cartoon_data, config['batch_size'])
  _, sketch_test_dataloader = get_data_loaders(sketch_data, config['batch_size'])
  no_dann_cartoon = alexdann(pretrained=True)
  criterion_cartoon, _, optimizer_cartoon, scheduler_cartoon = init_cnn_objects(no_dann_cartoon, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])
  no_dann_sketch = alexdann(pretrained=True)
  criterion_sketch, _, optimizer_sketch, scheduler_sketch = init_cnn_objects(no_dann_sketch, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])

  cartoon_losses, cartoon_accuracy_max = simple_train_test(no_dann_cartoon, source_dataloader, cartoon_test_dataloader, 
                                                           criterion_cartoon, optimizer_cartoon, scheduler_cartoon, max_epoch=config['epochs'])
  sketch_losses, sketch_accuracy_max = simple_train_test(no_dann_sketch, source_dataloader, sketch_test_dataloader, 
                                                         criterion_sketch, optimizer_sketch, scheduler_sketch, max_epoch=config['epochs'])

  curr_result = {'params': config, 'cartoon_losses': cartoon_losses, 'sketch_losses': sketch_losses, 
                 'best_acc': np.mean([cartoon_accuracy_max, sketch_accuracy_max])}
  results.append(curr_result)
  
best_conf = max(results, key=lambda x: x['best_acc'])
print(f"Best configuration is:\n{best_conf['params']}")
print(f"Highest mean accuracy: {best_conf['best_acc']}")


Starting epoch 1/10, LR = [0.001]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0, Loss_train 2.2247133255004883
Step 10, Loss_train 0.3557354509830475


100%|██████████| 19/19 [00:04<00:00,  3.97it/s]


Current Accuracy: 0.27099609375
Starting epoch 2/10, LR = [0.001]
Step 20, Loss_train 0.10233084112405777


...

100%|██████████| 31/31 [00:07<00:00,  4.10it/s]

Current Accuracy: 0.60205078125
Starting epoch 10/10, LR = [0.0003125]


Step 120, Loss_train 0.002783454954624176


100%|██████████| 31/31 [00:07<00:00,  4.09it/s]

Current Accuracy: 0.599609375
Best configuration is:
{'alpha': 'dynamic', 'batch_size': 128, 'epochs': 10, 'gamma': 0.5, 'lr': 0.005, 'step_size': 2}
Highest mean accuracy: 0.496826171875


In [0]:
params = best_conf['params']

source_dataloader, _ = get_data_loaders(source_data, batch_size=params['batch_size'])
_, test_dataloader = get_data_loaders(target_data, batch_size=params['batch_size'])
no_dann = alexdann(pretrained=True)
criterion, _, optimizer, scheduler = init_cnn_objects(no_dann, lr=params['lr'], step_size=params['step_size'], gamma=params['gamma'])

losses_no_dann, accuracy_max = simple_train_test(no_dann, source_dataloader, test_dataloader, criterion, optimizer, scheduler, max_epoch=params['epochs'])
print(f"Photo transfer to Art accuracy no DANN: {accuracy_max}")

Starting epoch 1/10, LR = [0.005]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0, Loss_train 2.5042428970336914
Step 10, Loss_train 0.10183731466531754


100%|██████████| 16/16 [00:04<00:00,  3.81it/s]


Current Accuracy: 0.4033203125
Starting epoch 2/10, LR = [0.005]
Step 20, Loss_train 0.0935046598315239


...

100%|██████████| 16/16 [00:04<00:00,  3.82it/s]

Current Accuracy: 0.501953125
Starting epoch 10/10, LR = [0.0003125]


Step 120, Loss_train 0.004503034055233002


100%|██████████| 16/16 [00:04<00:00,  3.86it/s]

Current Accuracy: 0.50048828125
Photo transfer to Art accuracy no DANN: 0.5166015625


**Training with DANN**

In [0]:
import math
def dann_train_test(model, source_dataloader, target_dataloader, test_dataloader, 
                    class_criterion, domain_criterion, optimizer, scheduler, max_epoch=NUM_EPOCHS, alpha=ALPHA, file_path=BASE_FILE_PATH, device=DEVICE):
    class_losses_y = []
    domain_losses_d = []
    accuracies = []
    loss_min = -1
    accuracy_max = 0
    count_diverge = 0
    
    model = model.to(device)
    cudnn.benchmark
    
    current_step = 0
    for epoch in range(max_epoch):
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, max_epoch, scheduler.get_lr()))
        class_loss, domain_loss, current_step = dann_train_src_target(model, source_dataloader, 
                                          target_dataloader, optimizer, class_criterion, domain_criterion, current_step,
                                          epoch, max_epoch, alpha=alpha, device=device)
        
        if math.isnan(class_loss) or math.isnan(domain_loss):
          count_diverge += 1
          if count_diverge >= 3:
            print("EARLY STOPPING")
            break;
        class_losses_y.append(class_loss)
        domain_losses_d.append(domain_loss)

        """if loss_min == -1 or loss_min > class_loss:
            loss_min = class_loss
            torch.save(model, f"{file_path}_best_model_dann.pth")"""
        accuracy = test_target(model, test_dataloader, class_criterion, device) / float(len(target_data))
        accuracies.append(accuracy)
        print(f"Accuracy on test set: {accuracy}")
        if accuracy_max < accuracy:
          accuracy_max = accuracy
          torch.save(model, f"{file_path}_best_model_dann.pth")
        scheduler.step()
        
    return class_losses_y, domain_losses_d, accuracies

In [0]:
params = {
    "lr": [1e-3, 5e-3],
    "batch_size": [128],
    "epochs": [10],
    "step_size": [2, 3],
    "gamma": [0.5],
    "alpha": ['dynamic']
}
grid = ParameterGrid(params)
dann_results = []
for config in grid:
  source_dataloader, _ = get_data_loaders(source_data, config['batch_size'])
  cartoon_dataloader, cartoon_test_dataloader = get_data_loaders(cartoon_data, config['batch_size'])
  sketch_dataloader, sketch_test_dataloader = get_data_loaders(sketch_data, config['batch_size'])
  dann_cartoon = alexdann(pretrained=True)
  criterion_cartoon_src, criterion_cartoon_tgt, optimizer_cartoon, scheduler_cartoon = init_cnn_objects(dann_cartoon, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])
  dann_sketch = alexdann(pretrained=True)
  criterion_sketch_src, criterion_sketch_tgt, optimizer_sketch, scheduler_sketch = init_cnn_objects(dann_sketch, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])

  class_losses_cartoon, domain_losses_cartoon, accuracies_cartoon = dann_train_test(
      dann_cartoon, source_dataloader, cartoon_dataloader, cartoon_test_dataloader,
      criterion_cartoon_src, criterion_cartoon_tgt, optimizer_cartoon, scheduler_cartoon, max_epoch=config['epochs'], alpha=config['alpha']
  )

  class_losses_sketch, domain_losses_sketch, accuracies_sketch = dann_train_test(
      dann_sketch, source_dataloader, sketch_dataloader, sketch_test_dataloader,
      criterion_sketch_src, criterion_sketch_tgt, optimizer_sketch, scheduler_sketch, max_epoch=config['epochs'], alpha=config['alpha']
  )

  curr_result = {'params': config, 'cartoon_losses': (class_losses_cartoon, domain_losses_cartoon), 'sketch_losses': (class_losses_sketch, domain_losses_sketch),
                 'best_acc': np.mean([max(accuracies_cartoon), max(accuracies_sketch)])}
  dann_results.append(curr_result)
  
best_conf = max(dann_results, key=lambda x: x['best_acc'])
print(f"Best configuration is:\n{best_conf['params']}")
print(f"Highest mean accuracy: {best_conf['best_acc']}")


Starting epoch 1/10, LR = [0.001]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0
Class Loss 2.159154176712036, Domain Loss 3.7594082355499268
Step 10
Class Loss 0.45151984691619873, Domain Loss 0.2007991373538971


100%|██████████| 19/19 [00:05<00:00,  3.38it/s]


Accuracy on test set: 0.28857421875
Starting epoch 2/10, LR = [0.001]
Step 20
Class Loss 0.1243690550327301, Domain Loss 0.045085158199071884


...

100%|██████████| 31/31 [00:09<00:00,  3.10it/s]

Accuracy on test set: 0.376953125
Starting epoch 10/10, LR = [0.0003125]


Step 120
Class Loss nan, Domain Loss nan


100%|██████████| 31/31 [00:09<00:00,  3.11it/s]

Accuracy on test set: 0.376953125
Best configuration is:
{'alpha': 'dynamic', 'batch_size': 128, 'epochs': 10, 'gamma': 0.5, 'lr': 0.005, 'step_size': 2}
Highest mean accuracy: 0.616943359375


In [0]:
params = best_conf['params']

source_dataloader, _ = get_data_loaders(source_data, params['batch_size'])
target_dataloader, test_dataloader = get_data_loaders(target_data, params['batch_size'])

dann = alexdann(pretrained=True)
class_criterion, domain_criterion, optimizer, scheduler = init_cnn_objects(dann, lr=params['lr'], step_size=params['step_size'],
                                                                           gamma=params['gamma'])
class_losses_y, domain_losses_d, accuracies = dann_train_test(dann, source_dataloader, target_dataloader,
                          test_dataloader, class_criterion, domain_criterion, optimizer, scheduler, max_epoch=params['epochs'], alpha=params['alpha'])

print(f"Photo transfer to Art accuracy DANN: {max(accuracies)}")

Starting epoch 1/10, LR = [0.005]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0
Class Loss 2.2315292358398438, Domain Loss 4.035977840423584
Step 10
Class Loss 0.13638703525066376, Domain Loss 0.2569566071033478


100%|██████████| 16/16 [00:06<00:00,  2.67it/s]


Accuracy on test set: 0.47216796875
Starting epoch 2/10, LR = [0.005]
Step 20
Class Loss 0.10701119899749756, Domain Loss 0.753000020980835


...

100%|██████████| 16/16 [00:05<00:00,  3.00it/s]

Accuracy on test set: 0.44970703125
Starting epoch 10/10, LR = [0.0003125]


Step 120
Class Loss 0.04244048893451691, Domain Loss 0.4897080063819885


100%|██████████| 16/16 [00:05<00:00,  3.06it/s]

Accuracy on test set: 0.4462890625
Photo transfer to Art accuracy DANN: 0.5322265625
